# Denoising

Given a noisy image, we want to find a denoised version of the image. In this part of the project, You will do the following:-

-  From true images, create a noisy training dataset
-  Visualize Different types of noisy images and comment on the differences
-  Using different models find denoised images.
-  Visualize noisy and their denoised counter parts for atleast 2-5 input samples for analysis.
-  Analyze which models give good results and why ?

*** Extra Python libraries(Pytorch, pandas, csv) are needed for this assignment. So please ensure the cell directly below excutes properly before proceeding with the assignment. If it doesn't please make sure to install those libraries before hand. ***

*** If you don't have access to GPU on your computer then install a CPU version of the pytorch library.  Required comments are provided to run the code on a CPU machine. Training on CPU is consideraly slower, so we would advise  you to use CADE machine for this assignment. CADE machine as 4GB GPU's which are enough for this project. ***

In [2]:
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import random
import skimage as ski
from skimage import io, transform
import csv
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
import sys
sys.path.insert(1, '../../utils')
from utils import NoiseDatsetLoader

# Adding Noise in the Images And Creating a Noisy Dataset

In [ ]:
def noiseAddtion(image,noiseIndicator):
    # This function gets an image of size m x n. Range of values is (0,1).
    # noiseIndicator is type of noise that needs to be added to the image
    # noiseIndicator == 0 indicates an addition of gaussian noise with mean 0 and var 0.08
    # noiseIndicator == 1 indicates an addition of Salt and Pepper noise with intensity variation of 0.08
    # noiseIndicator == 2 indicates an addition of poisson noise
    # noiseIndicator == 3 indicates an addition of Speckle noise of mean 0 and var 0.05
    
    ## This function should return a noisy version of the input image
    
    ##  ***************** Your Code starts here ***************** ##
    
    
    

    ## ***************** Your Code ends here ***************** ##
    return noisy

In [ ]:
def writeCSV(csv_file_name, Trainingdictlist):
    csv_columns = ['RefImageName','NoiseType','NoisyImage']
    try:
        with open(csv_file_name, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in Trainingdictlist:
                writer.writerow(data)
    except IOError:
        print("I/O error")

def createNoisyDataset(root_dir,train,numberOfSamples,noisyIndicatorLow,noisyIndicatorHigh):
    if(train==1):
        name_csv = pd.read_csv('../../utils/file_name_train.csv')
        csv_file_name = "TrainingDataSet.csv"
        directoryName = "TrainingDataset/"
        Trainingdictlist = [dict() for x in range(numberOfSamples)]
        if not os.path.exists(directoryName):
            os.makedirs(directoryName)
    else:
        name_csv = pd.read_csv('../../utils/file_name_test.csv')
        csv_file_name = "TestingDataSet.csv"
        directoryName = "TestingDataset/"
        Trainingdictlist = [dict() for x in range(numberOfSamples)]
        if not os.path.exists(directoryName):
            os.makedirs(directoryName)

    for i in range(numberOfSamples):
        r2 = random.randint(0,len(name_csv)-1) # Choose an image randomly from the dataset
        # Read the image from a path
        img_name = os.path.join(root_dir,name_csv.iloc[r2, 0])
        image    = io.imread(img_name)

        # Normalize the image to range (0,1) 
        M,N      = image.shape
        maximumPixelValue = np.max(image)
        image = image/maximumPixelValue

        # Choosing the noise randomly 
        r1 = random.randint(noisyIndicatorLow,noisyIndicatorHigh)
        noisyImage = noiseAddtion(image,r1)
        Trainingdictlist[i]={'RefImageName':img_name,'NoiseType': r1,'NoisyImage':str(i)+'.png'}
        io.imsave(directoryName+str(i)+'.png',noisyImage)
    writeCSV(csv_file_name, Trainingdictlist)

## The function below calls the above function to create a noisy training and testing dataset. 

Don't worry about lossy conversion warnings.

In [ ]:
createNoisyDataset('../../utils/GrayScale',1,400,0,5) ## Creating 400 samples of Training Data
createNoisyDataset('../../utils/GrayScale',0,200,0,5) ## Creating 200 samples of Training Data

## Visualize Samples of Each of the noise Type here Using any plotting library

Hint:- You can use Subplots to display different noisy images

In [ ]:
## Your Code Starts here


## Your Code ends here

## Comment on the Differences of different sample of noisy images

- Here

# Neural Network for Denoising

We now have a noisy training set and a reference images.

In [ ]:
dtype = torch.float32 ## instialize the data types used in training
cpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Complete the Training Function Below

In [ ]:
def trainingLoop(dataloader,model,optimizer,nepochs):
    ## This function Trains your model for 'nepochs'
    ## Using optimizer and model your passed as arguments
    ## On the data present in the DataLoader class passed
    ##
    ## This function the final loss value

    model = model.to(device=cpu)
    
    ## Our Loss function for this exercise is fixed to MSELoss
    loss_function = nn.MSELoss()
    loss_array =[]
    for e in range(nepochs):
            print("Epoch", e)
            for t, temp in enumerate(dataloader):
                ## Before Training Starts, put model in Training Mode
                model.train()
                NoisyImage = temp['NoisyImage'].to(device=cpu,dtype=dtype)
                referenceImage = temp['image'].to(device=cpu,dtype=dtype)
                ## Pass your input images through the model
                ## Be sure to set the gradient as Zero in Optmizer before backward pass. Hint:- zero_grad()
                ## Step the otimizer on after backward pass
                ## Assign the value computed by the loss function to a varible named 'loss'

                ## Due to dataset being Gray you may have to use unsqueeze function here

                ## ************* Start of your Code *********************** ##
                
                
                

                ## ************ End of your code ********************   ##
                loss_array.append(loss.cpu().detach().numpy())
            print("Training loss: ",loss)
    return loss, loss_array

In [ ]:
TrainingSet = NoiseDatsetLoader(csv_file='TrainingDataSet.csv', root_dir_noisy='TrainingDataset')
TestingSet  = NoiseDatsetLoader(csv_file='TestingDataSet.csv' , root_dir_noisy='TestingDataset')

## Batch Size is a hyper parameter, You may need to play with this paramter to get a more better network
## This is a user controlled parameter you can change this value
batch_size=16

## DataLoader is a pytorch Class for iterating over a dataset
dataloader_train  = DataLoader(TrainingSet,batch_size=batch_size,num_workers=4)
dataloader_test   = DataLoader(TestingSet,batch_size=1)

## Model 1 

- Declare a model with one conv2d filter with 1 input channel and output channel

In [ ]:
## ************* Start of your Code *********************** ##

model = 

## ************ End of your code ********************   ##


In [ ]:
## Set your hypter parameters
learningRate =  # Set this value between 1e-2 to 1e-4
weightDecay  =  # Set this value between 1e-2 to 1e-4

epochs =  # Start with 50 or 100 and can experiment with value as high as 300-400.

## Optimizer
## Please Declare An Optimizer for your model. We suggest you use SGD
## ************* Start of your Code *********************** ##

optimizer =

## ************ End of your code ********************   ##

In [ ]:
valMSE, loss_array = trainingLoop(dataloader_train,model,optimizer,epochs)

### Plot the graph of loss vs epoch

In [ ]:
## ************* Start of your Code *********************** ##



## ************ End of your code ********************   ##

## Complete the Accuracy Function Below 

In [ ]:
def checkTestingAccuracy(dataloader,model):
    ## This Function Checks Accuracy on Testing Dataset for the model passed
    ## This function should return the loss the Testing Dataset

    ## Before running on Testing Dataset the model should be in Evaluation Mode  
    
    model.eval()
    totalLoss = []
    loss_mse = nn.MSELoss()
    for t, temp in enumerate(dataloader):
        NoisyImage = temp['NoisyImage'].to(device=cpu,dtype=dtype)
        referenceImage = temp['image'].to(device=cpu,dtype=dtype)

        ## For Each Test Image Calculate the MSE loss with respect to Reference Image 
        ## Return the mean the total loss on the whole Testing Dataset
        ## ************* Start of your Code *********************** ##
        
        
        ## ************ End of your code ********************   ##

In [ ]:
## Test Your Model. Complete the implementation of checkTestingAccuracy function above 
model.eval()
testMSE = checkTestingAccuracy(dataloader_test,model)
print("Mean Square Error for the testing Set for the trained model is ", testMSE)

## Plot some of the Testing Dataset images by passing them through the trained model

In [ ]:
## ************* Start of your Code *********************** ##


## ************ End of your code ********************   ##

## Comment on the Denoised image Obtained

- Here

# Model 2

- Declare a model with five conv2d filters, with input channel size of first filter as 1 and output channel size of last filter as 1. 
- All other intermediate channels you can change as you see fit( use a maximum of 8 or 16 channel inbetween layers, otherwise the model might take a huge amount of time to train). 
- Add batchnorm2d layers between each convolution layer for faster convergence.

In [ ]:
## ************* Start of your Code *********************** ##

model2 =

## ************ End of your code ********************   ##

In [ ]:
## Set your hypter parameters
learningRate =   # Set this value between 1e-2 to 1e-4
weightDecay  =   # Set this value between 1e-2 to 1e-4

epochs =  # Start with 50 or 100 Can experiment with maximum 200 or 400 iterations

## Optimizer
## Please Declare An Optimizer for your model. We suggest you use SGD
## ************* Start of your Code *********************** ##

optimizer =

## ************ End of your code ********************   ##

In [ ]:
valMSE, loss_array = trainingLoop(dataloader_train,model2,optimizer,epochs)

In [ ]:
## Test Your Model
model2.eval()
testMSE = checkTestingAccuracy(dataloader_test,model2)
print("Mean Square Error for the testing Set for the trained model is ", testMSE)

## Plot some of the Testing Dataset images by passing them through the trained model

In [ ]:
## ************* Start of your Code *********************** ##



## ************ End of your code ********************   ##

# Model 3

- Add Non Linear activations((preferably ReLU) Between conv2d layers from Model 2

In [ ]:
## ************* Start of your Code *********************** ##

model3 =

## ************* End of your Code *********************** ##

In [ ]:
## Set your hypter parameters
learningRate =   # Set this value between 1e-2 to 1e-4
weightDecay  =   # Set this value between 1e-2 to 1e-4

epochs =  # Start with 50 or 100 Can experiment with maximum 200 or 400 iterations

## Optimizer
## Please Declare An Optimizer for your model. We suggest you use SGD
## ************* Start of your Code *********************** ##
optimizer = 
## ************ End of your code ********************   ##

In [ ]:
valMSE, loss_array = trainingLoop(dataloader_train,model3,optimizer,epochs)

In [ ]:
## Test Your Model
model3.eval()
testMSE = checkTestingAccuracy(dataloader_test,model3)
print("Mean Square Error for the testing Set for the trained model is ", testMSE)

## Plot some of the Testing Dataset images by passing them through the trained model

In [ ]:
## ************* Start of your Code *********************** ##


## ************ End of your code ********************   ##

# Analysis

Now that you have all the denoised outputs from different models, Answer the following questions.

a. Which Model performed best and why ??